In [1]:
import Pkg
Pkg.activate("Boris")

  Activating project at `~/Documents/Courses/Fall2024/Seminar/BorisPusher/Boris`


In [2]:
using Plots

In [20]:
include("integrators.jl")

boris_twoPA (generic function with 1 method)

In [4]:
# let's start with one particle of unit mass and charge
x_0 = [1/3, 1/4, 1/2];

# and initial velocity
v_0 = [2/5, 2/3, 1];

In [5]:
# numerical parameters
t0 = 0;
tf = 1;

# The errors in x and vk, v⊥ against different epsilon = 1/2^j are displayed in
# Fig. 8.1, where j = 4, . . . , 13.
j = 5;
epsilon = (1/2)^j/2;

# h = epsilon, 4*epsilon, 16*epsilon
h = epsilon;
nt = Int((tf - t0) / h + 1)

println("epsilon = ", epsilon)
println("h = ", h)
println("nt = ", nt)

epsilon = 0.015625
h = 0.015625
nt = 65


In [11]:
x_tBEA, v_tBEA = boris_expA(x_0, v_0, (t0, tf), nt, epsilon);

In [16]:
x_tBIA, v_tBIA = boris_impA(x_0, v_0, (t0, tf), nt, epsilon);

In [21]:
x_tT, v_tT = boris_twoPA(x_0, v_0, (t0, tf), nt, epsilon)

([0.3333333333333333 0.3316882103227664 … 0.3266809843224786 0.324007565512176; 0.25 0.23693812593788194 … 0.2527775720915222 0.23981888430784704; 0.5 0.5155700116830771 … 1.424216315452496 1.4379412603563126], [0.028062500000000046 -0.10528787267628031 … -0.02861898799761494 -0.171098803859364; 0.8438333333333333 -0.8359599399755551 … 0.8528106470152456 -0.8293560181552102; 0.9982638888888888 0.9964807477169395 … 0.8797743003608436 0.8783964738442578])

In [ ]:
x_tSB, v_tSB = boris(x_0, v_0, (t0, tf), nt, epsilon);
x_tBEA, v_tBEA = boris_expA(x_0, v_0, (t0, tf), nt, epsilon);
x_tBIA, v_tBIA = boris_impA(x_0, v_0, (t0, tf), nt, epsilon);
x_tBT, v_tBT = boris_twoPA(x_0, v_0, (t0, tf), nt, epsilon);
x_tRK, v_tRK = runge_kutta(x_0, v_0, (t0, tf), nt, epsilon);

In [ ]:
println(x_tSB[1, 3])
println(x_tBEA[1, 3])
println(x_tBIA[1, 3])
println(x_tRK[1, 3])

In [ ]:
plot(x_tBIA[1, :], x_tBIA[2, :], x_tBIA[3, :], label="Boris ImpA")
plot!(x_tRK[1, :], x_tRK[2, :], x_tRK[3, :], label="RK")
plot!(x_tBEA[1, :], x_tBEA[2, :], x_tBEA[3, :], label="Boris ExpA")
# plot!(x_tSB[1, :], x_tSB[2, :], x_tSB[3, :], label="Standard Boris")

In [ ]:
sqrt.(sum((x_tBIA .- x_tRK).^2, dims=1))[:]

In [ ]:
sqrt.(sum((x_tBEA .- x_tRK).^2, dims=1))[:]

In [ ]:
sqrt.(sum((x_tBIA .- x_tBEA).^2, dims=1))[:]

In [ ]:
# make log
plot(sqrt.(sum((x_tBIA .- x_tBEA).^2, dims=1))[:])
# # make log
# plot!(abs.(x_tBIA[2, :]-x_tBEA[2, :]))
# # make log
# plot!(abs.(x_tBIA[3, :]-x_tBEA[3, :]))

In [ ]:
# The errors in x and vk, v⊥ against different epsilon = 1/2^j are displayed in
# Fig. 8.1, where 

# array to store the errors
error_SB = [];
error_BEA = [];
error_BIA = [];
epsilons = [];
for j in 4:13
    epsilon = (1/2)^j;
    push!(epsilons, epsilon)

    # h = epsilon, 4*epsilon, 16*epsilon
    h = epsilon;
    nt = Int((tf - t0) / h + 1)
    println("nt = ", nt)

    x_tSB, v_tSB = boris(x_0, v_0, (t0, tf), nt, epsilon);
    x_tRK, v_tRK = runge_kutta(x_0, v_0, (t0, tf), nt, epsilon);
    x_tBEA, v_tBEA = boris_expA(x_0, v_0, (t0, tf), nt, epsilon);
    x_tBIA, v_tBIA = boris_impA(x_0, v_0, (t0, tf), nt, epsilon);

    # mean square error
    push!(error_SB, sum(sqrt.(sum((x_tRK .- x_tSB).^2, dims=1)))/nt)
    push!(error_BEA, sum(sqrt.(sum((x_tRK .- x_tBEA).^2, dims=1)))/nt)
    push!(error_BIA, sum(sqrt.(sum((x_tRK .- x_tBIA).^2, dims=1)))/nt)

end
println("j = ", j)


For the numerical results to provide a good approximation to the trajectory we require that the difference. where is some defined error tolerance, at each solution point. This difference is called the global error and is the accumulated error over all solution steps

In [ ]:
# log-log plot
plot(epsilons, error_SB, label="Boris", xscale=:log10, yscale=:log10, marker=:circle)
plot!(epsilons, error_BEA, label="Boris expA", xscale=:log10, yscale=:log10, marker=:circle)
plot!(epsilons, error_BIA, label="Boris impA", xscale=:log10, yscale=:log10, marker=:circle)

plot!(xlabel=s"$log_{10}(\epsilon)$", ylabel=s"$log_{10}($GE$)$")
plot!(title=s"The global errors of $x$ with $h=\epsilon$")